In [1]:
##Checking step by step before main script

In [2]:
#libraries
import math
import numpy as np
import scipy.io

In [3]:
#constants, may be repplaced with orekit

# considering arrays for conversion of matricies

#gps time, to start?
t_gps = np.array([0])
recef = np.zeros((1,3))
recef[0,0] = 6778137
# recef = np.array([6778137, 0, 0]) #(m), if this becomes 3D(?) will have to redo ecef_llh
vecef = np.array([0,0,7300]) #(m/2)
#MSIS placeholder
Nmsis = np.array([.5,.95,0,0,0,0]) #Number Densities (or ratio to total number density) of He, O, N2, O2, AR, H, N
Tmsis = 700 #temp at altitude (K)
Dmsis = 1e-12 #nuetral mass denisty (kg/m^3)

In [4]:
#Vector pointing from satellite to sun (this should be obtainable from Orekit, and in ECEF coords)
#but for now,
# approx. solar vector at 1 AU sun-earth distance and around Dec. solstice, at midnight UT
sun_deg = -23.45
sdeg1 = math.radians(sun_deg)
sdeg_array = np.zeros((1,3))
sdeg_array[0,0] = -math.cos(sdeg1)
sdeg_array[0,2] = math.sin(sdeg1)
# sdeg_array = np.array([-math.cos(sdeg1),0,math.sin(sdeg1)])

sunVector_ecef = 149597870700*sdeg_array - recef
#need to fix above # fixed
print(sunVector_ecef)

[[-1.37249017e+11  0.00000000e+00 -5.95322679e+10]]


In [5]:
#getareanrm function

def getareanrm():
    #data file with macromodel plate areas (m^2) and plate normal vectors in spacecraft body-fixed coordinates data refers to             #Starlink v1.0 and body-fixed coordinate system as shown in Starlink_geometry_20200331.jpg reflectivities and materials are           #guesses for now, satellite bus is assumed completely flat... need to revisit this at some point
    Au = 196.96655
    #SiO2 = 60.0843
    SiO2 = 20.03
    Al = 26.981538

    #arrays for returns
    A = np.zeros(6)
    plt_M = np.zeros(6)
    bwspec = np.zeros(6)
    bwdiff = np.zeros(6)
    nrm_sbf = np.zeros((3,6))


    #****SOLAR PANELS: Front panel -Y (Shark Fin mode) or -Z (Open Book mode, sada_pos=pi/2; default setting used here, to be changed     #outside of this routine)
    nrm_sbf[:,0] = [0, 0, -1] 
    A[0] = 8.05*2.77
    bwspec[0] = 0.05
    bwdiff[0] = 0.3
    plt_M[0] = SiO2

    #****SOLAR PANELS: Back panel +Y (Shark Fin mode) or +Z (Open Book mode, sada_pos=0; default setting used here, to be changed         #outside of this routine)
    nrm_sbf[:,1] = [0, 0, 1] 
    A[1] = 8.05*2.77
    bwspec[1] = 0.2
    bwdiff[1] = 0.4
    plt_M[1] = Al
    
    #****BUS: Zenith-facing -Z
    nrm_sbf[:,2] = [0, 0, 1] 
    A[2] = 2.82*1.27
    bwspec[2] = 0.2
    bwdiff[2] = 0.4
    plt_M[2] = Al 

    #***BUS: Nadir-facing +Z
    nrm_sbf[:,3] = [0, 0, -1] 
    A[3] = 2.82*1.27
    bwspec[3] = 0.2
    bwdiff[3] = 0.4
    plt_M[3] = Al    

    #****VISORS: Satellite-facing +Y
    nrm_sbf[:,4] = [0, 1, 0] 
    A[4] = .681+.426
    bwspec[4] = 0.2
    bwdiff[4] = 0.4
    plt_M[4] = Al  

    #****VISORS: Outward-facing -Y
    nrm_sbf[:,5] = [0, -1, 0] 
    A[5] = .681+.426
    bwspec[5] = 0.2
    bwdiff[5] = 0.4
    plt_M[5] = Al     

    for i in range(len(nrm_sbf)):
        nrm_sbf[:,i] = nrm_sbf[:,i] / np.linalg.norm(nrm_sbf[:,i])


    return  [nrm_sbf,A,plt_M,bwspec,bwdiff]

    #finished, should be good for script, works for following cell

In [6]:
#substitute for struct in matlab
satprop = []

[nrm_sbf,area,plt_M,bwspec,bwdiff] = getareanrm()
scmass = 260 #Starlink v1.0 mass (kg) this is from wikipedia(!)

satprop.append(nrm_sbf)
satprop.append(area)
satprop.append(plt_M)
satprop.append(bwspec)
satprop.append(bwdiff)
satprop.append(scmass)

# print(satprop)

In [7]:
# load matfile and access names for use 
att_model = scipy.io.loadmat('attitude_model.mat')
att_model.keys()
mean_slonsbf = att_model['mean_slonsbf']
# mean_slonsbf[30,30]
mean_ssada = att_model['mean_ssada']
# mean_ssada[30,30]

In [8]:
#2nd function ecef_llh

def ecef_llh(r_ecef,converge):
    #lat, lon in degrees
    #height in km
    a = 6378.1370; # km
    b = 6356.75231425; # km
    e = math.sqrt((a**2 - b**2)/a**2); # km

    x = r_ecef[:,0] # km, first value
    y = r_ecef[:,1] # km, second value
    z = r_ecef[:,2] # km, third value
    
    lon = math.atan2(y,x)
    p = math.sqrt(x**2 + y**2)
    lat = math.atan(z/(p*(1-e**2)))

    error = converge + 1

    while error > converge:

        N       = a**2/math.sqrt((a**2)*(math.cos(lat))**2 + (b**2)*(math.sin(lat))**2)
        height  = p/math.cos(lat) - N
        latn_1  = lat
        lat     = math.atan(z/(p*(1-(e**2)*N/(N + height))))
        error   = (abs(lat - latn_1)) #max taken out, might be needed later

    lat = lat*180/math.pi
    lon = lon*180/math.pi
    height = height

    return[lat,lon,height]



In [9]:
#setup attitude model
[lat,lon,height] = ecef_llh(1e-3*recef,1e-12)
# print(lat,lon,height)
#seems to work good, reasonable answer

slt = (t_gps/3600 + lon/15) % 24 #approx. local time
latedges = np.arange(-54, 54, 1)
sltedges = np.arange(0,24,.25)


latbin = np.digitize(lat,latedges)
sltbin = np.digitize(slt,sltedges)
print(sltbin)

#should be good, not sure if it works with multiple timesteps yet

[1]


In [10]:
#allocate variables

ecef2sbf = np.zeros((3,3,len(t_gps)))
vrel_sbf = np.zeros((3,len(t_gps)))
Cd = np.zeros((1,len(t_gps)))
Aref = np.zeros((1,len(t_gps)))
# print(Cd, Aref)

In [11]:
#attitude_model_rotmat function
def attitude_model_rotmat(recef,sunVector_ecef,solar_lon_sbf):
    # %   Calculates the ECEF=>SBF rotation matrix based on
    # % recef (position in ECEF coordinates), sunVector_ECEF 
    # % (vector pointing at the sun in ECEF coordinates), and
    # % solar_lon_sbf (the longitude of the sun wrt the SBF coordinates
    # % in the SBF-x and -y plane
    # %   The method transforms this information into a rotation matrix by
    # % constructing a set of coordinates 't' where t1 is zenith facing, 
    # % t2, is perpendicular to t1 and the sunVector, and t3 completes the 
    # % right-handed coordinate frame. The t=>SBF and t=>ECEF rotations are
    # % constructed and then combined to give ECEF=>SBF

    tsbf = np.zeros((3,3))
    tecef = np.zeros((3,3))

    tsbf[:,0] = [0,0,-1] #% t1 = zenith
    # tsbf[:,1] = np.cross(tsbf[:,1],[cosd(solar_lon_sbf(1)),sind(solar_lon_sbf(1)),0]) #% t2 = perpendicular to t1 and sunVector
    tsbf[:,1] = np.cross(tsbf[:,0],[math.cos(math.radians(solar_lon_sbf(1))),math.sin(math.radians(solar_lon_sbf(1))),0])
    tsbf[:,1] = tsbf[:,1]/math.sqrt(sum(tsbf[:,1]**2,1)) # normalize
    tsbf[:,2] = np.cross(tsbf[:,0],tsbf[:,1]) #% t3 = completes the right-hand coordinate

    # tecef = np.zeros((3,3))
    tecef[:,0] = recef[:,0]/math.sqrt(sum(recef[:,0]**2,1)) #% t1 = zenith
    tecef[:,1] = np.cross(tecef[:,1],sunVector_ecef[0,:]) #% t2 = perpendicular to t1 and sunVector
    tecef[:,1] = tecef[:,1]/math.sqrt(sum(tecef[:,1]**2,1)) # normalize
    tecef[:,2] = np.cross(tecef[:,0],tecef[:,1]) #% t3 = completes the right-hand coordinate

    ecef2sbf = tsbf*(np.conj(tecef))
    print(ecef2sbf(type))
    return [ecef2sbf]

In [12]:
tsbf = np.zeros((3,3))
tsbf[:,0] = [0,0,-1]
assis = np.cross([0,0-1],[math.cos(math.radians(30)),math.sin(math.radians(6)),0])
tsbf[:,1] = assis
tsbf[:,1] = tsbf[:,1]/math.sqrt(sum(tsbf[:,1]**2,1)); # normalize
tsbf[:,2] = np.cross(tsbf[:,0],tsbf[:,1]) #% t3 = completes the right-hand coordinate

In [13]:
tsbf = np.zeros((3,3))
tecef = np.zeros((3,3))
 
tecef[:,0] = recef[:,0]/math.sqrt(sum(recef[:,0]**2,1)) #% t1 = zenith
tecef[:,1] = np.cross(tecef[:,1],sunVector_ecef[0,:]) #% t2 = perpendicular to t1 and sunVector
tecef[:,1] = tecef[:,1]/math.sqrt(sum(tecef[:,1]**2,1)); # normalize
tecef[:,2] = np.cross(tecef[:,0],tecef[:,1]) #% t3 = completes the right-hand coordinate
ecef2sbf = tsbf*(np.conj(tecef))
print(ecef2sbf)

[[ 0. -0.  0.]
 [ 0.  0. -0.]
 [ 0.  0.  0.]]


In [14]:
# Loop thru example values (here we only have one data point though)
ecef2sbf = np.zeros((3,3,len(t_gps)))
sada_pos_interp = np.zeros((1,len(t_gps)))

for j in range(len(t_gps)):
	# % Lookup attitude model to calculate ECEF->SBF rotation matrix when quaternion interpolation is unreliable
	ecef2sbf[:,:,j] = attitude_model_rotmat(recef[:,j],sunVector_ecef[:,j],mean_slonsbf[sltbin[j],latbin[j]])
	sada_pos_interp[j] = mean_ssada[sltbin[j],latbin[j]]

	# % Calculate velocity of satellite relative to atmosphere (ignoring winds but including co-rotation) in SBF coords
	vrel_sbf[:,j] = ecef2sbf[:,:,j]*vecef[:,j]


    # % Orient solar panel based on sada_pod angle (in radians)
	# % (this won't work with parfor loops; instead modify the inputs to calc_srp below)
	# %satprop.nrm_sbf (:,1) = [ 0; -cos(sada_pos_interp(j)); -sin(sada_pos_interp(j))]; % solar panels
	# %satprop.nrm_sbf (:,2) = [ 0;  cos(sada_pos_interp(j));  sin(sada_pos_interp(j))]; % back of solar panels




IndexError: invalid index to scalar variable.

In [15]:
ecef2sbf = np.zeros((3,3,len(t_gps)))
sada_pos_interp = np.zeros((1,len(t_gps)))

for j in range(len(t_gps)):
	# % Lookup attitude model to calculate ECEF->SBF rotation matrix when quaternion interpolation is unreliable
	inky = attitude_model_rotmat(recef[:,j],sunVector_ecef[:,j],mean_slonsbf[sltbin[j],latbin[j]])
	# sada_pos_interp[j] = mean_ssada[sltbin[j],latbin[j]]

	# % Calculate velocity of satellite relative to atmosphere (ignoring winds but including co-rotation) in SBF coords
	# vrel_sbf[:,j] = ecef2sbf[:,:,j]*vecef[:,j]


    # % Orient solar panel based on sada_pod angle (in radians)
	# % (this won't work with parfor loops; instead modify the inputs to calc_srp below)
	# %satprop.nrm_sbf (:,1) = [ 0; -cos(sada_pos_interp(j)); -sin(sada_pos_interp(j))]; % solar panels
	# %satprop.nrm_sbf (:,2) = [ 0;  cos(sada_pos_interp(j));  sin(sada_pos_interp(j))]; % back of solar panels

print(inky)

IndexError: invalid index to scalar variable.